In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
import easyocr
import os

# --- 1. CONFIGURACIÓN INICIAL ---

# Carga el modelo YOLOv8 preentrenado para personas y vehículos
# (Puedes usar yolov11n.pt si lo tienes, yo usaré yolov8n.pt como ejemplo)
model_general = YOLO('yolov8n.pt')

# Carga tu modelo YOLO personalizado entrenado para detectar matrículas
# ¡¡IMPORTANTE: Reemplaza esto con la ruta a tu propio modelo entrenado!!
try:
    model_matriculas = YOLO('best.pt')  # Carga el 'best.pt' de tu entrenamiento
except Exception as e:
    print(f"Error cargando el modelo de matrículas 'best.pt'.")
    print("Asegúrate de haberlo entrenado y colocado en la misma carpeta.")
    print("Continuando sin detección de matrículas...")
    model_matriculas = None

# Inicializa el lector de OCR (EasyOCR)
# 'es' para español, 'en' para inglés. Puedes añadir más idiomas.
ocr_reader = easyocr.Reader(['es'], gpu=True) # Poner gpu=False si no tienes GPU o da problemas

# Clases de interés del modelo COCO (model_general)
# 0: 'person', 2: 'car', 3: 'motorcycle', 5: 'bus', 7: 'truck'
clases_interes = [0, 2, 3, 5, 7]
clases_nombres = {0: 'persona', 2: 'vehiculo', 3: 'vehiculo', 5: 'vehiculo', 7: 'vehiculo'}

# --- 2. PREPARACIÓN DE VÍDEO Y ARCHIVOS ---

# Ruta al vídeo de entrada (¡cámbiala por tu vídeo!)
video_path_in = 'video_test.mp4' # Reemplaza con la ruta a tu vídeo
video_path_out = 'resultado_video.mp4'
csv_path_out = 'resultados_deteccion.csv'

# Abre el vídeo de entrada
cap = cv2.VideoCapture(video_path_in)
if not cap.isOpened():
    print(f"Error: No se pudo abrir el vídeo de entrada en {video_path_in}")
    exit()

# Obtiene propiedades del vídeo para el escritor de salida
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Crea el objeto VideoWriter para guardar el vídeo de salida
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_path_out, fourcc, fps, (frame_width, frame_height))

# --- 3. PROCESAMIENTO Y SEGUIMIENTO ---

# Lista para almacenar todos los resultados para el CSV
resultados_csv = []

# Contadores
frame_num = 0
counts = {'persona': 0, 'vehiculo': 0}
tracked_ids = set() # Para contar objetos únicos

print("Iniciando procesamiento de vídeo...")

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame_num += 1

    # Realiza el seguimiento (tracking) con el modelo general
    # El tracker por defecto es BoT-SORT, como se menciona en la práctica
    results_general = model_general.track(
        frame,
        persist=True,
        classes=clases_interes,
        verbose=False # Poner True para más detalles
    )

    # Dibuja las cajas y obtiene los resultados
    frame_annotated = results_general[0].plot()

    # Procesa los resultados del tracking
    if results_general[0].boxes.id is not None:
        boxes = results_general[0].boxes.xyxy.cpu().numpy().astype(int)
        confidences = results_general[0].boxes.conf.cpu().numpy()
        class_ids = results_general[0].boxes.cls.cpu().numpy().astype(int)
        track_ids = results_general[0].boxes.id.cpu().numpy().astype(int)

        for i in range(len(boxes)):
            x1, y1, x2, y2 = boxes[i]
            conf = confidences[i]
            cls_id = class_ids[i]
            track_id = track_ids[i]

            # Nombre del objeto
            tipo_objeto = clases_nombres.get(cls_id, 'desconocido')

            # Contar objetos únicos
            if track_id not in tracked_ids:
                tracked_ids.add(track_id)
                counts[tipo_objeto] += 1

            # --- Detección de Matrícula (si es un vehículo) ---
            matricula_conf = None
            mx1, my1, mx2, my2 = None, None, None, None
            texto_matricula = ""

            if tipo_objeto == 'vehiculo' and model_matriculas is not None:
                # 1. Recorta la zona del vehículo
                vehiculo_crop = frame[y1:y2, x1:x2]

                if vehiculo_crop.size > 0:
                    # 2. Busca matrículas en el recorte del vehículo
                    results_matricula = model_matriculas.predict(vehiculo_crop, verbose=False)

                    # Si encuentra alguna matrícula (coge la de mayor confianza)
                    if len(results_matricula[0].boxes) > 0:
                        best_matricula = results_matricula[0].boxes[0] # Coge la primera/mejor
                        matricula_conf = best_matricula.conf.cpu().numpy()[0]
                        
                        # Coordenadas relativas al recorte
                        m_rel = best_matricula.xyxy.cpu().numpy().astype(int)[0]
                        
                        # Coordenadas absolutas (en el frame completo)
                        mx1, my1 = x1 + m_rel[0], y1 + m_rel[1]
                        mx2, my2 = x1 + m_rel[2], y1 + m_rel[3]

                        # 3. Recorta la matrícula para el OCR
                        matricula_crop = vehiculo_crop[m_rel[1]:m_rel[3], m_rel[0]:m_rel[2]]

                        if matricula_crop.size > 0:
                            # 4. Ejecuta OCR en el recorte de la matrícula
                            ocr_result = ocr_reader.readtext(matricula_crop, detail=0, allowlist='0123456789BCDFGHJKLMNPQRSTVWXYZ')
                            
                            if ocr_result:
                                texto_matricula = " ".join(ocr_result).replace(" ", "") # Une posibles fragmentos

                        # Dibuja la caja de la matrícula y el texto
                        cv2.rectangle(frame_annotated, (mx1, my1), (mx2, my2), (0, 255, 0), 2)
                        cv2.putText(frame_annotated, texto_matricula, (mx1, my1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


            # Almacena los datos para el CSV
            fila_csv = {
                'fotograma': frame_num,
                'tipo_objeto': tipo_objeto,
                'confianza': conf,
                'identificador_tracking': track_id,
                'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2,
                'matricula_en_su_caso': 1 if mx1 is not None else 0,
                'confianza_matricula': matricula_conf,
                'mx1': mx1, 'my1': my1, 'mx2': mx2, 'my2': my2,
                'texto_matricula': texto_matricula
            }
            resultados_csv.append(fila_csv)

    # --- 4. GUARDAR FRAME Y MOSTRAR PROGRESO ---

    # Muestra los contadores en el vídeo
    cv2.putText(frame_annotated, f"Personas: {counts['persona']}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(frame_annotated, f"Vehiculos: {counts['vehiculo']}", (50, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Escribe el frame anotado en el vídeo de salida
    out.write(frame_annotated)

    # Opcional: mostrar el vídeo en tiempo real
    # cv2.imshow("Práctica 4 - Detección y Seguimiento", frame_annotated)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #    break
    
    if frame_num % 100 == 0:
        print(f"Procesado fotograma {frame_num}...")


# --- 5. FINALIZACIÓN ---

print("Procesamiento de vídeo finalizado.")

# Libera los recursos de vídeo
cap.release()
out.release()
# cv2.destroyAllWindows()

# Genera el archivo CSV
print(f"Guardando resultados en {csv_path_out}...")
df = pd.DataFrame(resultados_csv)

# Reordena las columnas para que coincidan con la práctica
columnas_ordenadas = [
    'fotograma', 'tipo_objeto', 'confianza', 'identificador_tracking', 
    'x1', 'y1', 'x2', 'y2', 
    'matricula_en_su_caso', 'confianza_matricula', 'mx1', 'my1', 'mx2', 'my2', 'texto_matricula'
]
# Asegura que todas las columnas existan, rellenando con NaT/None si faltan
df = df.reindex(columns=columnas_ordenadas)

df.to_csv(csv_path_out, index=False)

print("--- Resumen de Conteo ---")
print(f"Total de Personas únicas detectadas: {counts['persona']}")
print(f"Total de Vehículos únicos detectados: {counts['vehiculo']}")
print("¡Práctica completada!")